In [1]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
%matplotlib inline

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 600})
plt.rcParams['lines.linewidth'] = 0.5 

In [113]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-8h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2021-01-01 00:00:00,1609459200000,28923.63,29470.00,28690.17,29092.83,18869.366827,2021-01-01 00:00:00,20210101
2021-01-01 08:00:00,1609488000000,29092.84,29600.00,28872.24,29188.67,20078.654834,2021-01-01 08:00:00,20210101
2021-01-01 16:00:00,1609516800000,29188.67,29360.00,28624.57,29331.69,15234.903350,2021-01-01 16:00:00,20210101
2021-01-02 00:00:00,1609545600000,29331.70,29820.50,28946.53,29750.00,17258.671371,2021-01-02 00:00:00,20210102
2021-01-02 08:00:00,1609574400000,29751.47,31800.00,29473.91,31691.29,43255.778007,2021-01-02 08:00:00,20210102
...,...,...,...,...,...,...,...,...
2023-05-11 00:00:00,1683763200000,27598.74,27622.68,27367.76,27450.07,9819.590800,2023-05-11 00:00:00,20230511
2023-05-11 08:00:00,1683792000000,27450.07,27630.14,26952.00,27179.42,19518.716970,2023-05-11 08:00:00,20230511
2023-05-11 16:00:00,1683820800000,27179.43,27219.52,26702.05,26968.62,18297.005880,2023-05-11 16:00:00,20230511


### Explore ranges

In [2]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-4h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2021-01-01 00:00:00,1609459200000,28923.63,29470.00,28690.17,29278.40,11560.456553,2021-01-01 00:00:00,20210101
2021-01-01 04:00:00,1609473600000,29278.41,29395.00,28806.54,29092.83,7308.910274,2021-01-01 04:00:00,20210101
2021-01-01 08:00:00,1609488000000,29092.84,29402.57,28872.24,29313.49,8283.705319,2021-01-01 08:00:00,20210101
2021-01-01 12:00:00,1609502400000,29313.49,29600.00,29030.14,29188.67,11794.949515,2021-01-01 12:00:00,20210101
2021-01-01 16:00:00,1609516800000,29188.67,29360.00,28624.57,29029.04,9850.965345,2021-01-01 16:00:00,20210101
...,...,...,...,...,...,...,...,...
2023-05-12 00:00:00,1683849600000,26968.61,27091.12,26528.01,26584.32,9615.458560,2023-05-12 00:00:00,20230512
2023-05-12 04:00:00,1683864000000,26584.33,26710.00,26120.74,26311.52,15964.270710,2023-05-12 04:00:00,20230512
2023-05-12 08:00:00,1683878400000,26311.51,26437.00,26234.79,26348.01,6428.017950,2023-05-12 08:00:00,20230512


In [115]:
from minizinc import Instance, Model, Solver, Status

import nest_asyncio
nest_asyncio.apply()

strategy = 'ranges'
strategy_params_json = f'./strategy-params/{strategy}.json'


fibs = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

matplotlib.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
dlen = len(odf)

sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=50,description='Rng',continuous_update=False)
sl_box = IntRangeSlider(value=[5, 10],min=1,max=50,step=1,description='Box',continuous_update=False)
sl_vola = IntRangeSlider(value=[50, 100],min=0,max=200,step=5,description='Vola',continuous_update=False)

strategy_params = {
    "box": {'wdg': sl_box},
    "vola": {'wdg': sl_vola}
}
strategy_params = {"wrange": {'wdg': sl_range}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

wdf = None
bmin, bmax = None, None
vmin, vmax = None, None
result = None
rstart, rend = None, None
def printer(wrange, box, vola):
    global wdf
    global bmin, bmax, vmin, vmax
    global result, rstart, rend
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    wstart, wend = wrange
    bmin, bmax = box
    vmin, vmax = vola

    wdf = odf.iloc[wstart:wend, :].copy()

    print(f'N={len(wdf)}\n')


    wdf['lclmin'] = wdf.close.rolling(bmin).min()
    wdf['lclmax'] = wdf.close.rolling(bmin).max()
    wdf = wdf.dropna()
    wdf['brng'] = ((wdf.lclmax / wdf.lclmin - 1) * 1000).astype(int)
    
    wdf['idx'] = range(len(wdf))
    wdf['inrange'] = (wdf.brng >= vmin) & (wdf.brng <= vmax)

    # fl_rng_rstart = wdf.inrange & ~wdf.inrange.shift().fillna(False)

    # rstart = wdf[fl_rng_rstart].idx.values

    ################################
    ### Call Minizinc solver
    ##------------------------------

    model_params = {
        'bmin': bmin, 
        'bmax': bmax,
        'nfl': len(wdf),
        # 'nst' : len(rstart),
        # 'start': (rstart + 1).tolist(), 
        'flags': wdf.inrange.astype(int).tolist()
        }
    with open(f'{strategy}-dzn.json', 'w+') as f: f.write(json.dumps(model_params, indent=4))

    #######

    # print('Model params:', model_params)

    mzmodel = Model("./ranges.mzn")
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, mzmodel)

    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()
    print(result.status)

    rstart, rend = None, None
    arymin, arymax = None, None
    if result.status != Status.UNSATISFIABLE:
        # [srstart, srend] =  [result['srstart'], result['srend']]
        rstart, rend = list(map(lambda s: sorted(list(map(int, s.split(',')))), result['_output_item'].replace('sres=[{', '').replace('}]', '').split('}, {')))

        rstart = np.array(rstart) - 1
        rend = np.array(rend) - 1
        arymin = [wdf.iloc[rstart[i]:rend[i]].close.min() for i in list(range(len(rstart)))]
        arymax = [wdf.iloc[rstart[i]:rend[i]].close.max() for i in list(range(len(rstart)))]
    print(f'rstart={rstart}\nrend={rend}\n')

    fig = plt.figure(figsize=(9,5))
    ax1 = fig.add_subplot(2,1,1)
    # f, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]})

    # ax = fig.add_subplot(2,1,1)
    arxmin = wdf.index[rstart]
    arxmax = wdf.index[rend]
    rfibs = [ 
        round(lmin + f*(lmax-lmin) ) for f in fibs
        for lmin,lmax in [
            [wdf.lclmin.values[rstart[i]], wdf.lclmax.values[rstart[i]]]
            for i in range(len(rstart))
        ]
    ]
    fibxmin = wdf.index[rstart.tolist() * len(fibs)]
    fibxmax = wdf.index[rend.tolist() * len(fibs)]
    
    wdf.close.plot(ax=ax1)
    # ax1.vlines(wdf.index[rstart], ymin=arymin, ymax=arymax, colors='red', lw=0.5)
    # ax1.vlines(wdf.index[rend], ymin=arymin, ymax=arymax, colors='red', lw=0.5)
    for v in wdf.index[rstart]: ax1.axvline(v, c='green', lw=0.4, linestyle='--')
    for v in wdf.index[rend]: ax1.axvline(v, c='green', lw=0.4, linestyle='--')
    # ax1.hlines(arymin, xmin=arxmin, xmax=arxmax , colors='orange', lw=0.2)
    # ax1.hlines(arymax, xmin=arxmin, xmax=arxmax , colors='orange', lw=0.2)
    ax1.hlines(wdf.lclmin.values[rstart], xmin=arxmin, xmax=arxmax , colors='green', lw=0.7, linestyles='--')
    ax1.hlines(wdf.lclmax.values[rstart], xmin=arxmin, xmax=arxmax , colors='green', lw=0.7, linestyles='--')
    wdf.lclmin.plot(lw=0.1, c='g', ax=ax1)
    wdf.lclmax.plot(lw=0.1, c='r', ax=ax1)
    ax1.hlines(rfibs, xmin=fibxmin, xmax=fibxmax, colors='gray', lw=0.4, linestyles='--')
    # wdf.brng.plot(ax=ax1, secondary_y=True, lw=0.07, c='maroon')
    # plt.grid()

    # plt.show();
    
if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);
plt.close()
# fig

Output()

In [87]:
# list(map(lambda s: sorted(list(map(int, s.split(',')))), result['_output_item'].replace('sres=[{', '').replace('}]', '').split('}, {')))
# result['nsn']
# [wdf[wdf.index[rstart[i]:rend[i]]].close.min() for i in list(range(len(rstart)))]

# [wdf.iloc[rstart[i]:rend[i]].close.min() for i in ymin, xmin=arxmin, xmax=arxmax , colors='orange', lw=0.2)
    # ax1.hlines(arlist(range(len(rstart)))]
# wdf.iloc[rstart[1]:rend[1]].close.min()
rfibs = [ 
    round(lmin + f*(lmin-lmax) ) for f in fibs
        for lmin,lmax in [
            [wdf.lclmin.values[rstart[i]], wdf.lclmax.values[rstart[i]]]
            for i in range(len(rstart))
    ]
 ]

wdf.index[rstart.tolist() * len(fibs)]

DatetimeIndex(['2022-06-28 16:00:00', '2022-07-01 20:00:00',
               '2022-07-06 08:00:00', '2022-07-09 12:00:00',
               '2022-07-12 12:00:00', '2022-07-13 00:00:00',
               '2022-07-15 00:00:00', '2022-07-16 12:00:00',
               '2022-06-28 16:00:00', '2022-07-01 20:00:00',
               '2022-07-06 08:00:00', '2022-07-09 12:00:00',
               '2022-07-12 12:00:00', '2022-07-13 00:00:00',
               '2022-07-15 00:00:00', '2022-07-16 12:00:00',
               '2022-06-28 16:00:00', '2022-07-01 20:00:00',
               '2022-07-06 08:00:00', '2022-07-09 12:00:00',
               '2022-07-12 12:00:00', '2022-07-13 00:00:00',
               '2022-07-15 00:00:00', '2022-07-16 12:00:00',
               '2022-06-28 16:00:00', '2022-07-01 20:00:00',
               '2022-07-06 08:00:00', '2022-07-09 12:00:00',
               '2022-07-12 12:00:00', '2022-07-13 00:00:00',
               '2022-07-15 00:00:00', '2022-07-16 12:00:00',
               '2022-06-